In [64]:
def composites(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups

    for phase in phases:
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        precip_values = [
            precip_ds.where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = [precip_time, precip_values]
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)

def is_seas(month, seas):
    if seas=='jja':
        return (month >= 6) & (month <= 8)
    elif seas=='djf':
        return (month >= 12) | (month <= 2)
    elif seas=='son':
        return (month >= 9) & (month <= 11)
    elif seas=='mam':
        return (month >= 3) & (month <= 5)
        
def placehold(array,fillval):
    if len(array)==0:
        return(fillval)
    else: return(array)

In [65]:
import xarray as xr
import pandas as pd
import numpy as np

seas = 'mam'
if seas=='mam':
    th = 20.02698
    seasname = "SHAutumn"
elif seas=='djf':
    th = 26.05894
    seasname = "SHSummer"

In [66]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/"

dirinc ="/Users/fadhlilrizki/Documents/PhD/thesis/dataout/"

diro = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/combined_wave/"

In [67]:
regions = ["TopEnd","CapeYork","Kimberley","Queensland"]
latSs   = [-17, -16, -20, -25]
latNs   = [-10, -10, -13.5, -18]
lonRs   = [137, 146, 128, 145]
lonLs   = [129, 141, 120, 140]

code = 1 #regioncode
if code==0:
    region ="TE"
elif code==1:
    region="CY"
elif code==2:
    region="KY"
elif code==3:
    region="QL"

In [68]:
fw_1 = "local_er_phase_"+seasname+".133.nc"
fw_2 = "local_kelvin_phase_"+seasname+".133.nc"
fw_3 = "local_mrg_phase_"+seasname+".133.nc"
fw_4 = "local_td_phase_"+seasname+".133.nc"
fw_5 = "local_mjo_phase_"+seasname+".133.nc"

fe_1 = "er_wave_extreme_"+region+"_"+seas+".nc"
fe_2 = "kelvin_wave_extreme_"+region+"_"+seas+".nc"
fe_3 = "mrg_wave_extreme_"+region+"_"+seas+".nc"
fe_4 = "td_wave_extreme_"+region+"_"+seas+".nc"
fe_5 = "mjo_wave_extreme_"+region+"_"+seas+".nc"


In [69]:
dw_1 = xr.open_dataset(diri+fw_1)
dw_2 = xr.open_dataset(diri+fw_2)
dw_3 = xr.open_dataset(diri+fw_3)
dw_4 = xr.open_dataset(diri+fw_4)
dw_5 = xr.open_dataset(diri+fw_5)

# dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [70]:
de_1 = xr.open_dataset(dirinc+fe_1)
de_2 = xr.open_dataset(dirinc+fe_2)
de_3 = xr.open_dataset(dirinc+fe_3)
de_4 = xr.open_dataset(dirinc+fe_4)
de_5 = xr.open_dataset(dirinc+fe_5)

# dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [71]:
erp  = de_1.composite.values
klvp = de_2.composite.values
mrgp = de_3.composite.values
tdp  = de_4.composite.values
mjop = de_5.composite.values

In [72]:
dw_1 = dw_1.sel(time=slice("1981-01-01","2018-12-31"))
dw_2 = dw_2.sel(time=slice("1981-01-01","2018-12-31"))
dw_3 = dw_3.sel(time=slice("1981-01-01","2018-12-31"))
dw_4 = dw_4.sel(time=slice("1981-01-01","2018-12-31"))
dw_5 = dw_5.sel(time=slice("1981-01-01","2018-12-31"))

# dprec = dprec.sel(time=slice("1981-01-01","2018-12-31"),
#                   lon=slice(lonLs[code],lonRs[code]), 
#                   lat=slice(latSs[code],latNs[code])
#                  )

# prec = dprec.precip
# prec

In [73]:
# weights = np.cos(np.deg2rad(prec.lat))
# weights.name = "weights"

# pr_weighted = prec.weighted(weights)
# praave = pr_weighted.mean(("lon", "lat"))

# praave

# prth = xr.where(praave==0, np.nan, praave)
# prth

In [74]:
er_active     = xr.where(dw_1["amp"] >= 1, dw_1, np.nan).dropna(dim="time")
kelvin_active = xr.where(dw_2["amp"] >= 1, dw_2, np.nan).dropna(dim="time")
mrg_active    = xr.where(dw_3["amp"] >= 1, dw_3, np.nan).dropna(dim="time")
td_active     = xr.where(dw_4["amp"] >= 1, dw_4, np.nan).dropna(dim="time")
mjo_active    = xr.where(dw_5["amp"] >= 1, dw_5, np.nan).dropna(dim="time")


In [75]:
# er_active.to_dataframe().to_csv('er_act.csv')

In [76]:
tc   = pd.read_csv('/Users/fadhlilrizki/Documents/PhD/thesis/data/TC_dates.csv',
                   usecols=["0"], parse_dates=["0"])

tc_da = xr.DataArray(tc)
er_active         = xr.where(~er_active['time'].isin(tc_da), er_active, np.nan).dropna(dim='time')
mrg_active        = xr.where(~mrg_active['time'].isin(tc_da), mrg_active, np.nan).dropna(dim='time')
kelvin_active     = xr.where(~kelvin_active['time'].isin(tc_da), kelvin_active, np.nan).dropna(dim='time')
td_active         = xr.where(~td_active['time'].isin(tc_da), td_active, np.nan).dropna(dim='time')
mjo_active        = xr.where(~mjo_active['time'].isin(tc_da), mjo_active, np.nan).dropna(dim='time')

er_active

<xarray.Dataset>
Dimensions:  (time: 2080)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-01 1981-03-02 ... 2018-05-09
Data variables:
    x        (time) float32 0.7357 0.9528 1.047 ... 0.7224 0.3687 -0.02076
    y        (time) float32 0.7333 0.4502 0.0409 ... -0.8386 -1.073 -1.031
    amp      (time) float32 1.039 1.054 1.048 1.077 ... 1.029 1.107 1.134 1.032
    phase    (time) float64 8.0 8.0 1.0 2.0 2.0 3.0 ... 8.0 1.0 2.0 2.0 3.0 3.0

In [77]:
er_size = er_active.groupby("phase").groups
key_to_value_lengths = {k:len(v) for k, v in er_size.items()}
key_to_value_lengths

{1.0: 274,
 2.0: 281,
 3.0: 258,
 4.0: 244,
 5.0: 282,
 6.0: 259,
 7.0: 238,
 8.0: 244}

In [78]:
er_comp  = {}
klv_comp = {}
mrg_comp = {}
td_comp  = {}
mjo_comp = {}

In [79]:
# praave = praave.sel(time=is_seas(praave['time.month'],seas))
# prth.quantile(0.95, method="linear", skipna=True)

In [80]:
er_comp

{}

In [81]:
er_comp = erp
# er_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in er_comp.items())

mrg_comp = mrgp
# mrg_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mrg_comp.items())

kelvin_comp = klvp
# kelvin_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in kelvin_comp.items())

td_comp = tdp
# td_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in td_comp.items())

mjo_comp = mjop
# mjo_comp_mean = dict((k, np.mean(v[1])/prob_season) for k, v in mjo_comp.items())

In [82]:
er_comp
td_comp
mjo_comp

array([0.18596405, 0.40962347, 0.5943557 , 0.8019085 , 0.8316088 ,
       2.670355  , 1.4633447 , 1.1630558 ], dtype=float32)

In [83]:

er_wets     = [int(k+1) for k,v in enumerate(erp) if v>=1.25]
kelvin_wets = [int(k+1) for k,v in enumerate(klvp) if v>=1.25]
mrg_wets    = [int(k+1) for k,v in enumerate(mrgp) if v>=1.25]
td_wets     = [int(k+1) for k,v in enumerate(tdp) if v>=1.25]
mjo_wets    = [int(k+1) for k,v in enumerate(mjop) if v>=1.25]

er_drys     = [int(k+1) for k,v in enumerate(erp) if v<=0.8]
kelvin_drys = [int(k+1) for k,v in enumerate(klvp) if v<=0.8]
mrg_drys    = [int(k+1) for k,v in enumerate(mrgp) if v<=0.8]
td_drys     = [int(k+1) for k,v in enumerate(tdp) if v<=0.8]
mjo_drys    = [int(k+1) for k,v in enumerate(mjop) if v<=0.8]

with open('phases_'+region+'_'+seas+'.txt', 'w') as f:
    f.write(str(["er-wet",er_wets,"dry", er_drys]))
    f.write("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
    f.write("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
    f.write("\n"+str(["td-wet",td_wets,"dry", td_drys]))
    f.write("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))
# er_wets

In [84]:

print(str(["er-wet",er_wets,"dry", er_drys]))
print("\n"+str(["kelvin-wet",kelvin_wets,"dry", kelvin_drys]))
print("\n"+str(["mrg-wet",mrg_wets,"dry", mrg_drys]))
print("\n"+str(["td-wet",td_wets,"dry", td_drys]))
print("\n"+str(["mjo-wet",mjo_wets,"dry", mjo_drys]))

['er-wet', [4, 5], 'dry', [2, 7, 8]]

['kelvin-wet', [], 'dry', [1, 3, 5, 6]]

['mrg-wet', [7], 'dry', [1]]

['td-wet', [1, 2], 'dry', []]

['mjo-wet', [6, 7], 'dry', [1, 2, 3]]


In [85]:
if seas=='djf':
    er_wet        = xr.where((er_active["phase"]==5) , er_active, np.nan).dropna(dim="time")
    kelvin_wet    = xr.where((kelvin_active["phase"]==2), kelvin_active, np.nan).dropna(dim="time") #gaada
    mrg_wet       = xr.where((mrg_active["phase"]==8), mrg_active, np.nan).dropna(dim="time") #gaada
    td_wet        = xr.where((td_active["phase"]==8) , td_active, np.nan).dropna(dim="time")#gaada
    mjo_wet       = xr.where(((mjo_active["phase"]>=5) & (mjo_active["phase"]<=7)), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_wet        = xr.where((er_active["phase"]>=4) & (er_active["phase"]<=5) , er_active, np.nan).dropna(dim="time")
    kelvin_wet    = xr.where((kelvin_active["phase"]==8), kelvin_active, np.nan).dropna(dim="time") #gaad a
    mrg_wet       = xr.where((mrg_active["phase"]==7) , mrg_active, np.nan).dropna(dim="time")
    td_wet        = xr.where((td_active["phase"]<=2), td_active, np.nan).dropna(dim="time")
    mjo_wet       = xr.where((mjo_active["phase"]>=6) & (mjo_active["phase"]<=7), mjo_active, np.nan).dropna(dim="time")

In [86]:
if seas=='djf':
    er_dry        = xr.where((er_active["phase"]==1) | (er_active["phase"]>=7), er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]==7) | (kelvin_active["phase"]==3), kelvin_active, np.nan).dropna(dim="time")
    mrg_dry       = xr.where(((mrg_active["phase"]<=2) | (mrg_active["phase"]>=7)), mrg_active, np.nan).dropna(dim="time")
    td_dry        = xr.where((td_active["phase"]==3) | (td_active["phase"]==4) | (td_active["phase"]==1) | (td_active["phase"]==7) , td_active, np.nan).dropna(dim="time")
    mjo_dry       = xr.where((mjo_active["phase"]<=4), mjo_active, np.nan).dropna(dim="time")
elif seas=='mam':
    er_dry        = xr.where((er_active["phase"]>=7) | (er_active["phase"]==2) , er_active, np.nan).dropna(dim="time")
    kelvin_dry    = xr.where((kelvin_active["phase"]==1) | (kelvin_active["phase"]==3) | (kelvin_active["phase"]==5) | (kelvin_active["phase"]==6)  , kelvin_active, np.nan).dropna(dim="time")
    mrg_dry       = xr.where((mrg_active["phase"]==1), mrg_active, np.nan).dropna(dim="time")
    td_dry        = xr.where((td_active["phase"]==8) , td_active, np.nan).dropna(dim="time") #gaada
    mjo_dry       = xr.where((mjo_active["phase"]<=3) , mjo_active, np.nan).dropna(dim="time")

In [87]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 526)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-08 1981-03-21 ... 2018-04-17
Data variables:
    x        (time) float32 -0.5555 -1.09 -1.713 -2.017 ... -2.506 -2.758 -2.763
    y        (time) float32 -1.055 -2.124 -1.339 ... -0.9103 -0.3713 0.4549
    amp      (time) float32 1.192 2.387 2.174 2.057 ... 2.387 2.666 2.783 2.8
    phase    (time) float64 4.0 4.0 4.0 5.0 5.0 4.0 ... 5.0 4.0 4.0 5.0 5.0 5.0

In [88]:
er_wet

<xarray.Dataset>
Dimensions:  (time: 526)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-08 1981-03-21 ... 2018-04-17
Data variables:
    x        (time) float32 -0.5555 -1.09 -1.713 -2.017 ... -2.506 -2.758 -2.763
    y        (time) float32 -1.055 -2.124 -1.339 ... -0.9103 -0.3713 0.4549
    amp      (time) float32 1.192 2.387 2.174 2.057 ... 2.387 2.666 2.783 2.8
    phase    (time) float64 4.0 4.0 4.0 5.0 5.0 4.0 ... 5.0 4.0 4.0 5.0 5.0 5.0

In [89]:
#ER AND KELVIN
er_er_wetwet  = er_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
er_er_drydry  = er_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

klv_klv_wetwet  = kelvin_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
klv_klv_drydry  = kelvin_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mrg_mrg_wetwet  = mrg_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mrg_drydry  = mrg_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

td_td_wetwet  = td_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
td_td_drydry  = td_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")

mjo_mjo_wetwet  = mjo_wet["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_mjo_drydry  = mjo_dry["time"].dt.strftime("%Y%m%d").to_dataframe(name="date")


#ER kelvin
er_klv_wetwet = kelvin_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drywet = kelvin_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_drydry = kelvin_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_klv_wetdry = kelvin_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MRG and KELVIN

mrg_klv_wetwet = kelvin_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drywet = kelvin_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_drydry = kelvin_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_klv_wetdry = kelvin_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#TD and Kelvin

td_klv_wetwet  = kelvin_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drywet  = kelvin_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_drydry  = kelvin_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_klv_wetdry  = kelvin_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

#MJO and Kelvin

mjo_klv_wetwet  = kelvin_wet["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drywet  = kelvin_wet["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_drydry  = kelvin_dry["time"].where(mjo_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mjo_klv_wetdry  = kelvin_dry["time"].where(mjo_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")



In [90]:
#ER AND MRG
er_mrg_wetwet = mrg_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drywet = mrg_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_drydry = mrg_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mrg_wetdry = mrg_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [91]:
#ER AND TD
er_td_wetwet = td_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drywet = td_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_drydry = td_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_td_wetdry = td_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [92]:
#ER AND MJO
er_mjo_wetwet = mjo_wet["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drywet = mjo_wet["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_drydry = mjo_dry["time"].where(er_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
er_mjo_wetdry = mjo_dry["time"].where(er_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [93]:
#mrg AND MJO
mrg_mjo_wetwet = mjo_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drywet = mjo_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_drydry = mjo_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_mjo_wetdry = mjo_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [94]:
#TD AND MJO
td_mjo_wetwet = mjo_wet["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drywet = mjo_wet["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_drydry = mjo_dry["time"].where(td_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
td_mjo_wetdry = mjo_dry["time"].where(td_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [95]:
#MRG AND TD
mrg_td_wetwet = td_wet["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drywet = td_wet["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_drydry = td_dry["time"].where(mrg_dry["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")
mrg_td_wetdry = td_dry["time"].where(mrg_wet["time"]).dt.strftime("%Y%m%d").to_dataframe(name="date")

In [96]:
er_er_wetwet

,date
time,
1981-03-08,19810308
1981-03-21,19810321
1981-03-22,19810322
1981-03-23,19810323
1981-03-24,19810324
...,...
2018-04-13,20180413
2018-04-14,20180414
2018-04-15,20180415


In [97]:
er_er_wetwet.to_csv(diro+"er_er_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_er_drydry.to_csv(diro+"er_er_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_klv_wetwet.to_csv(diro+"er_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_wetdry.to_csv(diro+"er_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_drywet.to_csv(diro+"er_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_klv_drydry.to_csv(diro+"er_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_mrg_wetwet.to_csv(diro+"er_mrg_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_wetdry.to_csv(diro+"er_mrg_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_drywet.to_csv(diro+"er_mrg_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_mrg_drydry.to_csv(diro+"er_mrg_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_td_wetwet.to_csv(diro+"er_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_td_wetdry.to_csv(diro+"er_td_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_td_drywet.to_csv(diro+"er_td_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_td_drydry.to_csv(diro+"er_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

er_mjo_wetwet.to_csv(diro+"er_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_wetdry.to_csv(diro+"er_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_drywet.to_csv(diro+"er_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
er_mjo_drydry.to_csv(diro+"er_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

In [98]:
klv_klv_wetwet.to_csv(diro+"klv_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
klv_klv_drydry.to_csv(diro+"klv_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_mrg_wetwet.to_csv(diro+"mrg_mrg_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mrg_drydry.to_csv(diro+"mrg_mrg_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_klv_wetwet.to_csv(diro+"mrg_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_wetdry.to_csv(diro+"mrg_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drywet.to_csv(diro+"mrg_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_klv_drydry.to_csv(diro+"mrg_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_td_wetwet.to_csv(diro+"mrg_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_wetdry.to_csv(diro+"mrg_td_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_drywet.to_csv(diro+"mrg_td_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_td_drydry.to_csv(diro+"mrg_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mrg_mjo_wetwet.to_csv(diro+"mrg_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_wetdry.to_csv(diro+"mrg_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drywet.to_csv(diro+"mrg_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mrg_mjo_drydry.to_csv(diro+"mrg_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

In [99]:
td_td_wetwet.to_csv(diro+"td_td_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_td_drydry.to_csv(diro+"td_td_drydry_CY_"+seas+".csv", columns=['date'], index = False)

td_klv_wetwet.to_csv(diro+"td_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_wetdry.to_csv(diro+"td_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_drywet.to_csv(diro+"td_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
td_klv_drydry.to_csv(diro+"td_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)

td_mjo_wetwet.to_csv(diro+"td_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_wetdry.to_csv(diro+"td_mjo_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_drywet.to_csv(diro+"td_mjo_drywet_CY_"+seas+".csv", columns=['date'], index = False)
td_mjo_drydry.to_csv(diro+"td_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)


In [100]:

mjo_mjo_wetwet.to_csv(diro+"mjo_mjo_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_mjo_drydry.to_csv(diro+"mjo_mjo_drydry_CY_"+seas+".csv", columns=['date'], index = False)

mjo_klv_wetwet.to_csv(diro+"mjo_klv_wetwet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_wetdry.to_csv(diro+"mjo_klv_wetdry_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drywet.to_csv(diro+"mjo_klv_drywet_CY_"+seas+".csv", columns=['date'], index = False)
mjo_klv_drydry.to_csv(diro+"mjo_klv_drydry_CY_"+seas+".csv", columns=['date'], index = False)


In [101]:
er_td_drydry

,date
time,
1981-03-05,19810305
1981-05-09,19810509
1982-03-08,19820308
1983-03-16,19830316
1985-05-02,19850502
1986-03-11,19860311
1986-04-04,19860404
1987-04-22,19870422
1989-03-17,19890317


In [102]:
! ncl 'seas="{seas}"' code=1 composite_combined_area_wave_extremes_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr_aave_in
Type: float
Total Size: 55516 bytes
            13879 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 13879]
Coordinates: 
            time: [47847.375..61725.375]
Number Of Attributes: 14
  _FillValue :	1e+20
  standard_name :	lwe_thickness_of_precipitation_amount
  long_name :	Daily precipitation
  units :	mm
  grid_mapping :	spatial_ref
  least_significant_digit :	3
  cell_methods :	time: sum
  analysis_version_number :	3.01
  number_of_stations_reporting :	6600
  frequency :	daily
  length_scale_for_analysis :	  80
  coverage_content_type :	physicalMeasurement
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average

Variable: pr_aave
Type: float
Total Size: 55516 bytes
            13879 values
Number of D

In [103]:
! ncl 'seas="{seas}"' code=1 composite_combined_area_wave_forpy.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr
Type: float
Total Size: 57580 bytes
            14395 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 14395]
Coordinates: 
            time: [47847.375..62241.375]
Number Of Attributes: 14
  _FillValue :	1e+20
  coverage_content_type :	physicalMeasurement
  length_scale_for_analysis :	  80
  frequency :	daily
  number_of_stations_reporting :	6600
  analysis_version_number :	3.01
  cell_methods :	time: sum
  least_significant_digit :	3
  grid_mapping :	spatial_ref
  units :	mm
  long_name :	Daily precipitation
  standard_name :	lwe_thickness_of_precipitation_amount
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average


Variable: pr_Anom
Type: float
Total Size: 57580 bytes
            14395 values
Number of Dimensio

In [41]:
# ! ncl 'seas="mam"' code=1 composite_combined_area_wave_extremes_forpy.ncl